In [1]:
from package import Excel_Read,Time_Deal,Email_Custom,Pdf_Attach,Picture_Attach,IP_Check,Functions
from email.utils import formataddr

In [2]:
#读取表格    
rawdata = Excel_Read.get_xlsdata("settings/setting_informations.xls")

#读取活页簿 
data_sheets = ['mail_content','mailto_lists','mails_lists','socks_ip']

mail_content,mailto_lists,mails_lists,socks_ip = [rawdata.sheet_by_name(name) for name in data_sheets]

In [3]:
mail_subject,mail_content,attach_pictures,attach_pdfs = mail_content.col_values(1)[0:4]

In [4]:
attach_pictures,attach_pdfs= [pic for pic in attach_pictures.split("#")],[pic for pic in attach_pdfs.split("#")],

In [5]:
mailto_lists = [mailto_lists.row_values(num) for num in range(1,mailto_lists.nrows)]

In [6]:
mails_lists =  [mails_lists.row_values(num) for num in range(1,mails_lists.nrows)]

In [7]:
socks_ip =  [socks_ip.row_values(num) for num in range(1,socks_ip.nrows)]

In [8]:
picture_attaches = [Picture_Attach.uploadPicture("attach_files/" + pic,'') for pic in attach_pictures]

In [9]:
pdf_attaches = [Pdf_Attach.uploadPdf("attach_files/" + pdf)  for pdf in attach_pdfs]

In [10]:
#发件正文内容进行判定，是发html文件，还是纯文本。
if mail_content == '{}{}':
    content = open("settings/content.html",encoding='utf-8').read().replace("\n",'')
    content_class = 'html'
else:
    content = mail_content
    content_class = 'plain'

In [30]:
user_monitor = Functions.Monitor_User(mails_lists)

In [56]:
user_monitor

{'jyzyg129@163.com': {'TotalTimes': 8,
  'Per_Count': 1,
  'Remain_Times': 8,
  'TimeGap_Lists': [2.464, 4.965, 3.01, 2.999, 2.62, 2.225, 3.839],
  'Last_SendTime': ''},
 '1021527932@qq.com': {'TotalTimes': 5,
  'Per_Count': 2,
  'Remain_Times': 5,
  'TimeGap_Lists': [116.55, 80.272, 116.398, 92.215],
  'Last_SendTime': ''},
 'jyzyg129@vip.163.com': {'TotalTimes': 3,
  'Per_Count': 1,
  'Remain_Times': 3,
  'TimeGap_Lists': [41.117, 22.491],
  'Last_SendTime': ''}}

In [ ]:
informations = []
for information in user_monitor.keys():
    Send_Times = user_monitor[information]['TotalTimes'] - user_monitor[information]['Remain_Times']
    informations.append([information,user_monitor[information]['Per_Count'] * Send_Times,user_monitor[information]['Remain_Times'],''])

In [57]:
input("本次发件完毕，请点击Enter键退出！")      


本次发件完毕，请点击Enter键退出！


''

In [61]:
'mnnnn' in vars()

False

In [ ]:
Functions.Record_CSV(file_records[2],['No','发件账号','发件成功地址数量','发件失败地址数量','配置发送次数剩余数量','备注'])

In [31]:
total_count,total_time = Functions.Check_Possiblity(user_monitor)

In [36]:
instructions = """

{}

发件配置策略评估：

1、总需求发件对象邮件地址数量：{}个；配置发件箱数量：{}个；
2、根据发件配置策略评估：
    总共可发件数量：{}个邮件地址，完成{}个地址发件总耗时约{}分钟；
{}

{}
"""
if total_count >= len(mailto_lists):
    conclusion = "\n结论：由于可发件数量大于目标发件数量，配置策略可行。请点击Enter键开始发件，如需重新配置发件策略，请直接关闭窗口！"
else :
    conclusion = "\n结论：由于可发件数量小于目标发件数量，配置策略建议优化。如需重新配置发件策略，请直接关闭窗口；如坚持采用该策略，请点击Enter键开始发件"

In [45]:
print(instructions.format('#'*100,len(mailto_lists),len(mails_lists),total_count,total_count,total_time,conclusion,'#'*100))



####################################################################################################

发件配置策略评估：

1、总需求发件对象邮件地址数量：4个；配置发件箱数量：3个；
2、根据发件配置策略评估：
    总共可发件数量：21个邮件地址，完成21个地址发件总耗时约8分钟；

结论：由于可发件数量大于目标发件数量，配置策略可行。请点击Enter键开始发件，如需重新配置发件策略，请直接关闭窗口！

####################################################################################################



In [40]:
'#',len(mailto_lists),len(mails_lists),total_count,total_time,conclusion,'#'

('#',
 4,
 3,
 21,
 8,
 '\n结论：由于可发件数量大于目标发件数量，配置策略可行。请点击Enter键开始发件，如需重新配置发件策略，请直接关闭窗口！',
 '#')

In [37]:
#开始邮件发送：

while mailto_lists:
    
    #循环检查每一个发件箱
    for mails in mails_lists:

        #发件人信息：发件地址，授权码，发件人昵称
        user,password,user_name = mails[1],mails[2],mails[3]
        
        if not user_monitor[user]['Remain_Times'] > 0: continue
        
        #判定是否符合发件条件
        #条件有两种情况：a、没发过，首次发；b、发过且时间间隔已到
        
        if user_monitor[user]['Last_SendTime'] == '' or (Time_Deal.time.time() - user_monitor[user]['Last_SendTime'] >= user_monitor[user]['TimeGap_Lists'][0]) :
            
            #发件人昵称处理
            user_name = formataddr([user_name,user])
            
            #发件箱单次发件收件人数量
            person_count = user_monitor[user]['Per_Count']
            #符合发件条件后，区分是单发还是群发；
            if  person_count == 1:

                #取出收件人信息，一个收件人
                customs_one = mailto_lists[0]
                
                #收件人处理，个性化收件人名称
                touser = [formataddr([customs_one[1],customs_one[2]])]
                
                #个性化邮件主题，添加变量
                custom_subject = mail_subject.format(*customs_one[3:5])
                #个性化正文内容，添加变量
                custom_content = Functions.Replace_strchar(content,'{}',customs_one[5:])

                #邮件正式开始发送
                Email_Custom.sendEmail(user.split('@')[-1],user_name,user,password,custom_subject,custom_content,content_class,touser,picture_attaches,pdf_attaches)

                print(Time_Deal.getTimeNow(),user,customs_one[2],"1对1发送成功！")

                user_monitor[user]['Remain_Times'] -= 1
                user_monitor[user]['Last_SendTime'] = Time_Deal.time.time()
                user_monitor[user]['TimeGap_Lists'].pop(0)
                mailto_lists = mailto_lists[1:]
            
            elif person_count > 1:
            
                #取出收件人信息，Percount个收件人
                customs_multiple = mailto_lists[0:person_count]
                
                #收件人处理，个性化收件人名称
                tousers = [formataddr([customs_one[1],customs_one[2]]) for customs_one in customs_multiple]
                
                #个性化邮件主题，添加变量
                custom_subject = mail_subject.format('','')
                #个性化正文内容，添加变量
                custom_content = Functions.Replace_strchar(content,'{}',['',''])

                #邮件正式开始发送
                Email_Custom.sendEmail(user.split('@')[-1],user_name,user,password,custom_subject,custom_content,content_class,tousers,picture_attaches,pdf_attaches)

                print(Time_Deal.getTimeNow(),user,"1对"+ str(person_count) + "发送成功！")

                user_monitor[user]['Remain_Times'] -= 1
                user_monitor[user]['Last_SendTime'] = Time_Deal.time.time()
                user_monitor[user]['TimeGap_Lists'].pop(0)
                mailto_lists = mailto_lists[person_count:]
                
            if not mailto_lists: break

2020-07-02 18:50:24 jyzyg129@163.com itsnick129@gmail.com 1对1发送成功！
2020-07-02 18:50:27 1021527932@qq.com 1对2发送成功！
2020-07-02 18:50:29 jyzyg129@vip.163.com jyzyg129@vip.163.com 1对1发送成功！


In [48]:
IP_Check.getcurrentip()

'117.150.165.237'

In [55]:
with open('data_record/error.txt' ,'a') as code:
    code.write('129')

In [62]:
Time_Deal.getTimeNow()

'2020-07-03 22:00:28'

In [75]:
# 初始化文档存储csv文件
file_record_names = ['发件成功记录','发件失败记录','邮件群发整体报告']
file_name_start_time = Time_Deal.getTimeNow()[0:-3].replace(':','时') + "分"
file_records = [file_name_start_time + "_"+name for name in file_record_names]

In [76]:
file_records

['2020-07-03 22时17分_发件成功记录',
 '2020-07-03 22时17分_发件失败记录',
 '2020-07-03 22时17分_邮件群发整体报告']

In [68]:
with open(file_records[0] + ".txt",'a') as code:
    code.write('ceshi')